## Reading sensor data from PlantBuddy.

### PlantBuddy class

In [10]:
import struct
import time
from collections import namedtuple
from typing import Callable
from bleak import BleakClient, BleakError, BleakScanner

# Create a PlantBuddy class that can be used to connect/disconnect from it and fetch data.
class PlantBuddy():
    # MAC Address of the PlantBuddy - Debug
    mac_addr = "F4:94:15:33:BC:8A"
    # MAC Address of the PlantBuddy - Release
    # mac_addr = "C9:D0:A2:60:FB:2F"
    # UUID's of service and sensordata characteristic
    uuid_service = "f4c2abcd-6e5f-48a2-b9b2-a4f762791d85"
    uuid_char_sensordata = "f4c20001-6e5f-48a2-b9b2-a4f762791d85"
    uuid_char_timestamp = "f4c20002-6e5f-48a2-b9b2-a4f762791d85"
    # Local variables
    ble_client = None
    callback = None

    def __init__(self):
        pass

    async def connect(self, timeout: float = 60.0):
        device = await BleakScanner.find_device_by_address(self.mac_addr, timeout=timeout)
        if not device:
            raise BleakError(f"A device with address {self.ble_address} could not be found.")
        # Save the client locally
        self.ble_client = BleakClient(device)
        await self.ble_client.connect(timeout=timeout)
        await self._update_timestamp()

    async def disconnect(self):
         await self.ble_client.disconnect()

    def install_callback(self, callback: Callable[[bytearray], None]):
        self.callback = callback

    async def start_listening(self):
        if self.callback is None:
            raise BleakError(f"No callback was specified!")
        await self.ble_client.start_notify(self.uuid_char_sensordata, self._notification_handler)

    async def stop_listening(self):
        await self.ble_client.stop_notify(self.uuid_char_sensordata)

    async def _update_timestamp(self):
        epoch_time = int(time.time())
        await self.ble_client.write_gatt_char(self.uuid_char_timestamp, epoch_time.to_bytes(4, 'little'))

    def _notification_handler(self, sender, data):
        Packet = namedtuple('Packet', 'timestamp soil_humidity luminous_flux air_humidity air_temperature battery_voltage')
        raw_packet = Packet._make(struct.unpack("<IIIHHHxx", data))
        phys_packet = raw_packet._replace(air_temperature=raw_packet.air_temperature/100, air_humidity=raw_packet.air_humidity/100)

        self.callback(phys_packet)


In [12]:
import asyncio

def new_data(sensor_data):
    # Data needs to be a dictionary containing all my items
    print(f"Received: {sensor_data}")

pb = PlantBuddy()
await pb.connect()  
pb.install_callback(new_data)
await pb.start_listening()
await asyncio.sleep(10.0)
await pb.stop_listening()


Received: Packet(timestamp=1619706188, soil_humidity=903298, luminous_flux=20, air_humidity=42.43, air_temperature=24.93, battery_voltage=3251)
Received: Packet(timestamp=1619706189, soil_humidity=903077, luminous_flux=20, air_humidity=42.45, air_temperature=24.95, battery_voltage=3251)
Received: Packet(timestamp=1619706190, soil_humidity=903107, luminous_flux=20, air_humidity=42.48, air_temperature=24.93, battery_voltage=3248)
Received: Packet(timestamp=1619706191, soil_humidity=903097, luminous_flux=20, air_humidity=42.42, air_temperature=24.92, battery_voltage=3244)
Received: Packet(timestamp=1619706192, soil_humidity=903097, luminous_flux=20, air_humidity=42.36, air_temperature=24.93, battery_voltage=3248)
Received: Packet(timestamp=1619706193, soil_humidity=903107, luminous_flux=20, air_humidity=42.28, air_temperature=24.91, battery_voltage=3244)
Received: Packet(timestamp=1619706194, soil_humidity=903097, luminous_flux=20, air_humidity=42.23, air_temperature=24.95, battery_voltag